In [1]:
import pandas as pd

df = pd.read_csv('data/solarpower.csv', sep=',')
df.head()

,date,cum_power
0,2017-01-01,20111.0
1,2017-01-02,20112.0
2,2017-01-03,20115.0
3,2017-01-04,20117.0
4,2017-01-05,20119.0


В нашем временном ряде есть несколько пропусков, они обозначены числом -1. Заполните пропуски средним арифметическим между двумя соседними наблюдениями.

In [8]:
for i, row in df.iterrows():
    if row['cum_power'] == -1:
        prev_val = df.loc[i-1, 'cum_power']
        next_val = df.loc[i+1, 'cum_power']
        df.at[i, 'cum_power'] = (prev_val + next_val) / 2

display(round(df['cum_power'].mean()))
display(df[df['cum_power'] == -1])

23848

,date,cum_power


Теперь, когда таблица заполнена, создайте столбец day_power — ежедневная выработка солнечной энергии.

Далее создайте данные о средней ежедневной выработке энергии на каждый месяц-год.

Совет: для того чтобы выделить из даты год и месяц, проще всего будет воспользоваться методом аксессора dt.to_period('M'), который позволяет выделять периоды (в данном случае 'М' — месяц).

Затем сгруппируйте данные по периодам (год-месяц) и рассчитайте среднее в столбце day_power.

Сколько солнечной энергии в среднем в день выработали в марте 2017 года? Ответ округлите до целого.

In [14]:
df['day_power'] = df['cum_power'].diff()
df['month'] = pd.to_datetime(df['date']).dt.to_period('M')
df['year'] = pd.to_datetime(df['date']).dt.to_period('Y')

df.head()

,date,cum_power,day_power,month,year
0,2017-01-01,20111.0,NaN,2017-01,2017
1,2017-01-02,20112.0,1.0,2017-01,2017
2,2017-01-03,20115.0,3.0,2017-01,2017
3,2017-01-04,20117.0,2.0,2017-01,2017
4,2017-01-05,20119.0,2.0,2017-01,2017


In [28]:
# Group by 'year' and 'month' and calculate mean of 'day_power'
grouped_df = df.groupby(['year', 'month'])['day_power'].mean().reset_index()
display(round(grouped_df))

,year,month,day_power
0,2017,2017-01,3.0
1,2017,2017-02,4.0
2,2017,2017-03,11.0
3,2017,2017-04,16.0
4,2017,2017-05,19.0
5,2017,2017-06,22.0
6,2017,2017-07,18.0
7,2017,2017-08,15.0
8,2017,2017-09,12.0
9,2017,2017-10,6.0


Постройте график временного ряда — средней ежедневной выработки солнечной энергии на каждый месяц-год. Не забудьте подписать оси и сам график.

In [30]:
import plotly.express as px

# avoiding "TypeError: Object of type Period is not JSON serializable"
grouped_df['month'] = grouped_df['month'].astype(str)

fig = px.line(grouped_df, x = 'month', y = 'day_power', title = 'Mean daily power per month')
fig.show()